## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"

## Этап 4. Сервис рекомендаций

__Постановка задачи__

Напишите FastAPI-микросервис для выдачи рекомендаций, который:
- принимает запрос с идентификатором пользователя и выдаёт рекомендации,
- учитывает историю пользователя,
- смешивает онлайн- и оффлайн-рекомендации.

В README опишите стратегию смешивания онлайн- и оффлайн-рекомендаций.

Выполните тестирование микросервиса в ноутбуке `part_3_test.ipynb`:
- для пользователя без персональных рекомендаций,
- для пользователя с персональными рекомендациями, но без онлайн-истории,
- для пользователя с персональными рекомендациями и онлайн-историей.

### Инициализация

In [ ]:
import numpy as np
import pandas as pd
import requests

In [ ]:
import sys

# Проверяем, в каком окружении работаем
print(sys.executable)

In [ ]:
# Адреса сервисов
features_store_url = "http://127.0.0.1:8010"
events_store_url = "http://127.0.0.1:8020"
recommendations_url = "http://127.0.0.1:8000"

# Общий для всех запросов заголовок
headers = {"Content-type": "application/json", "Accept": "text/plain"}

Для запуска сервисов с помощью uvicorn необходимо выполнить 3 команды в терминале, находясь в папке проекта:

```
uvicorn recommendations_service:app
uvicorn features_service:app --port 8010
uvicorn events_service:app --port 8020
```

### Загрузка данных

Загрузким треки и события для отображения рекомендаций со всеми полями

In [ ]:
events = pd.read_parquet("events.parquet")
items = pd.read_parquet("items.parquet")

### Получение рекомендаций по умолчанию

Напишем функцию для отображения данных

In [ ]:
def display_items(item_ids):

    item_columns_to_use = ["item_id", "name", "genres", "albums", "artists"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)

Отправляем запрос для получения рекомендаций по умолчанию

In [ ]:
params = {'k': 10}
resp = requests.post(recommendations_url + "/recommendations_default", headers=headers, params=params)

if resp.status_code == 200:
    recs = resp.json()
    item_ids = recs['recs']
    display_items(item_ids)
    
else:
    recs = []
    print(f"status code: {resp.status_code}")

In [ ]:
# Сравниваем с топ-10 треков из файла
top_k_pop_items = pd.read_parquet("top_popular.parquet")
top_k_pop_items.head(10)

### Получение персональных рекомендаций без онлайн-истории

In [ ]:
# Выберем произвольного пользователя случайным образом
user_id = events['user_id'].sample().iat[0]

In [ ]:
# Отправляем запрос для получения только оффлайн-рекомендаций
params = {'user_id': 0, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)

if resp.status_code == 200:
    recs = resp.json()
    item_ids = recs['recs']
    display_items(item_ids)
    
else:
    recs = []
    print(f"status code: {resp.status_code}")

### Получение персональных рекомендаций с онлайн-историей

Выберем 3 произвольных трека случайным образом и добавим их в онлайн-историю пользователя

In [ ]:
item_ids = items['items_id'].sample(3)

for item_id in item_ids:
    params = {"user_id": user_id, "item_id": item_id}
    resp = requests.post(events_store_url + "/put", headers=headers, params=params)
    if resp.status_code == 200:
        result = resp.json()
    else:
        result = None
        print(f"status code: {resp.status_code}")    
    print(result)

Получаем смешанные рекомендации. Оффлайн-рекомендации помещаем на четные места выходного списка, онлайн - на четные.

In [ ]:
params = {"user_id": user_id, 'k': 10}
resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)

if resp.status_code == 200:
    result = resp.json()
    item_ids = recs['recs']
    display_items(item_ids)
    
else:
    result = None
    print(f"status code: {resp.status_code}")